    Bastien KATUSZYNSKI - Maxime LEFEBVRE

## #1-Collecte des données

In [42]:
import sys
import random
import requests
import shutil
import os
import string
from pandas import json_normalize
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
import urllib.parse

endpoint_url = "https://query.wikidata.org/sparql"
# Get cities
query = """SELECT DISTINCT ?grandeville ?grandevilleLabel ?pays ?paysLabel ?image {
  ?grandeville wdt:P31 wd:Q1549591;
               wdt:P17 ?pays;
               wdt:P18 ?image.

  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],fr". }
}
LIMIT 100"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (
        sys.version_info[0],
        sys.version_info[1],
    )
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


array = []
results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    array.append(
        (
            result["grandevilleLabel"]["value"],
            result["paysLabel"]["value"],
            result["image"]["value"],
        )
    )

dataframe = pd.DataFrame(array, columns=["ville", "pays", "image"])
dataframe = dataframe.astype(
    dtype={"ville": "<U200", "pays": "<U200", "image": "<U200"}
)

def download_image(url):
    folder = "images/"
    headers = {"User-Agent": "Mozilla/5.0"}
    request = requests.get(url, allow_redirects=True, headers=headers, stream=True)
    if request.status_code == 200:
        # Extract the filename from the URL and decode it
        filename = urllib.parse.unquote(os.path.basename(url))

        # Remove any query parameters from the filename
        filename, _ = os.path.splitext(filename)
        
        # Remove any special characters from the filename
        valid_chars = "-_.() %s%s" % (string.ascii_letters, string.digits)
        filename = ''.join(c for c in filename if c in valid_chars)
        filename = "img_" + filename
        # Extract the extension from the URL and add it to the filename
        extension = os.path.splitext(url)[1]
        filename += extension
        with open(folder + filename, "wb") as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
    return request.status_code


dataframe.image.apply(download_image)

0     200
1     200
2     200
3     200
4     200
     ... 
95    200
96    200
97    200
98    200
99    200
Name: image, Length: 100, dtype: int64

In [43]:
from PIL import Image
import os
import json
import sys
import random
import requests
import shutil
import string
from pandas import json_normalize
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
import urllib.parse

# Chemin du dossier à parcourir
folder_path = "./images"

# Liste des propriétés de chaque image
images = []

# Parcours des fichiers du dossier
for filename in os.listdir(folder_path):
    filepath = os.path.join(folder_path, filename)
    if os.path.isfile(filepath):
        # Vérifie si le fichier est une image
        if filepath.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".gif")):
            # Ouverture de l'image
            with Image.open(filepath) as img:
                # Récupération des propriétés de l'image
                
                imgfile = Image.open(filepath)
            
                size = img.size
                format = img.format
                exif_data = imgfile._getexif()
                
                if exif_data is not None:
                    # Récupération des informations Exif spécifiques
                    image_width = exif_data.get(256)
                    image_height = exif_data.get(257)
                    orientation = exif_data.get(274)
                    creation_date = exif_data.get(306)
                    camera_model = exif_data.get(531)
                
                    # Ajout des propriétés à la liste des images
                    images.append({
                        "filename": filename,
                        "size": size,
                        "heigth": image_height,
                        "width": image_width,
                        "format": format,
                        "orientation": orientation,
                        "creation_date":creation_date,
                        "camera_model": camera_model
                    })
                    
                # Ajout des propriétés à la liste des images
                images.append({
                    "filename": filename,
                    "size": size,
                    "format": format
                })
                

with open("images_informations.json", "w") as f:
    json.dump(images, f, indent=4, ensure_ascii=False)

## #2-Etiquetage et Annotation

In [82]:
from PIL import Image
import numpy
import math
import matplotlib.pyplot as plot
from sklearn.cluster import KMeans
import os
import json
import sys
import random
import requests
import shutil
import string
from pandas import json_normalize
import pandas as pd
import webcolors
import multiprocessing


# Chemin du dossier à parcourir
folder_path = "./images"

# Nombre de processus en parallèle
num_processes = 4

# Liste des propriétés de chaque image
images = []
numberOfCluster = 20


def addTagsImage(imgfile, file_name, tags):
    # Ajouter des tags
    imgfile.info["keywords"] = tags
        
    # Enregistrer l'image avec les tags
    imgfile.save(os.path.join(folder_path,file_name))
                
def clearTagsImage(imgfile, file_name):
    if "keywords" in imgfile.info:
        print(imgfile.info["keywords"])
        del imgfile.info["keywords"]

    # Enregistrer l'image sans tags
    imgfile.save(os.path.join(folder_path,file_name))

# Calcul de distance des couleurs
def color_distance(color1, color2):
    r1, g1, b1 = color1
    r2, g2, b2 = color2
    return math.sqrt((r1 - r2) ** 2 + (g1 - g2) ** 2 + (b1 - b2) ** 2)

processes = []
file_array_process = [[] for _ in range(num_processes)]
file_number = 0 

# Parcours des fichiers du dossier
for filename in os.listdir(folder_path):
    filepath = os.path.join(folder_path, filename)
    
    if os.path.isfile(filepath):
        try :
            # Vérifie si le fichier est une image
            if filepath.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".gif")):
                # Ouverture de l'image
                with Image.open(filepath) as imgfile:
                    file_name = os.path.basename(filepath)

                    tags = ["ok"]

                    # Enregistrement de l'image avec les tags
                    addTagsImage(imgfile, file_name, ["ok"])
                    
                    imgfile.info["keywords"] = tags
                    # Enregistrer l'image avec les tags
                    imgfile.save(os.path.join(folder_path,file_name))
                    
                    # Vérifier si les tags ont été ajoutés avec succès
                    print("Tags ajoutés à l'image :", str(imgfile.info))

        except Exception as e:
            print("Erreur sur l'image " + filepath)
    
index = 0
# Parcours des fichiers du dossier
for filename in os.listdir(folder_path):
    filepath = os.path.join(folder_path, filename)

    # Vérifie si le fichier est une image
    if filepath.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".gif")):
        # Ouverture de l'image
        with Image.open(filepath) as imgfile:
                # Vérifier si les tags ont été ajoutés avec succès
                try:
                    print(str(index) + "_Tags de l'image" + filename + " : " + str(imgfile.info["keywords"]))
                except Exception as e:
                    print("Error reading tags" + str(e))
    index += 1

Tags ajoutés à l'image : {'jfif': 257, 'jfif_version': (1, 1), 'jfif_unit': 0, 'jfif_density': (1, 1), 'keywords': ['ok']}
Tags ajoutés à l'image : {'jfif': 257, 'jfif_version': (1, 1), 'jfif_unit': 0, 'jfif_density': (1, 1), 'keywords': ['ok']}
Tags ajoutés à l'image : {'jfif': 257, 'jfif_version': (1, 1), 'jfif_unit': 0, 'jfif_density': (1, 1), 'keywords': ['ok']}
Tags ajoutés à l'image : {'jfif': 257, 'jfif_version': (1, 1), 'jfif_unit': 0, 'jfif_density': (1, 1), 'keywords': ['ok']}
Tags ajoutés à l'image : {'jfif': 257, 'jfif_version': (1, 1), 'jfif_unit': 0, 'jfif_density': (1, 1), 'keywords': ['ok']}
Tags ajoutés à l'image : {'icc_profile': b'\x00\x00\x0f\x9cappl\x02\x10\x00\x00mntrRGB XYZ \x07\xe3\x00\x0b\x00\x02\x00\x11\x00 \x00-acspAPPL\x00\x00\x00\x00APPL\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf6\xd6\x00\x01\x00\x00\x00\x00\xd3-appl\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

Tags ajoutés à l'image : {'jfif': 257, 'jfif_version': (1, 1), 'jfif_unit': 0, 'jfif_density': (1, 1), 'keywords': ['ok']}
Tags ajoutés à l'image : {'jfif': 257, 'jfif_version': (1, 1), 'jfif_unit': 0, 'jfif_density': (1, 1), 'keywords': ['ok']}
Tags ajoutés à l'image : {'keywords': ['ok']}
Tags ajoutés à l'image : {'jfif': 257, 'jfif_version': (1, 1), 'jfif_unit': 0, 'jfif_density': (1, 1), 'keywords': ['ok']}
Tags ajoutés à l'image : {'jfif': 257, 'jfif_version': (1, 1), 'jfif_unit': 0, 'jfif_density': (1, 1), 'keywords': ['ok']}
Tags ajoutés à l'image : {'jfif': 257, 'jfif_version': (1, 1), 'jfif_unit': 0, 'jfif_density': (1, 1), 'keywords': ['ok']}
Tags ajoutés à l'image : {'jfif': 257, 'jfif_version': (1, 1), 'jfif_unit': 0, 'jfif_density': (1, 1), 'keywords': ['ok']}
Tags ajoutés à l'image : {'jfif': 257, 'jfif_version': (1, 1), 'jfif_unit': 0, 'jfif_density': (1, 1), 'keywords': ['ok']}
Tags ajoutés à l'image : {'jfif': 257, 'jfif_version': (1, 1), 'jfif_unit': 0, 'jfif_density'

Tags ajoutés à l'image : {'keywords': ['ok']}
Tags ajoutés à l'image : {'jfif': 257, 'jfif_version': (1, 1), 'jfif_unit': 0, 'jfif_density': (1, 1), 'keywords': ['ok']}
Tags ajoutés à l'image : {'jfif': 257, 'jfif_version': (1, 1), 'jfif_unit': 0, 'jfif_density': (1, 1), 'keywords': ['ok']}
Tags ajoutés à l'image : {'jfif': 257, 'jfif_version': (1, 1), 'jfif_unit': 0, 'jfif_density': (1, 1), 'keywords': ['ok']}
Tags ajoutés à l'image : {'jfif': 257, 'jfif_version': (1, 1), 'jfif_unit': 0, 'jfif_density': (1, 1), 'keywords': ['ok']}
Tags ajoutés à l'image : {'jfif': 257, 'jfif_version': (1, 1), 'jfif_unit': 0, 'jfif_density': (1, 1), 'keywords': ['ok']}
Tags ajoutés à l'image : {'jfif': 257, 'jfif_version': (1, 1), 'jfif_unit': 0, 'jfif_density': (1, 1), 'keywords': ['ok']}
Tags ajoutés à l'image : {'jfif': 257, 'jfif_version': (1, 1), 'jfif_unit': 0, 'jfif_density': (1, 1), 'keywords': ['ok']}
Tags ajoutés à l'image : {'icc_profile': b'\x00\x00\x0cHLino\x02\x10\x00\x00mntrRGB XYZ \x07\

Tags ajoutés à l'image : {'jfif': 257, 'jfif_version': (1, 1), 'jfif_unit': 0, 'jfif_density': (1, 1), 'keywords': ['ok']}
Tags ajoutés à l'image : {'jfif': 257, 'jfif_version': (1, 1), 'jfif_unit': 0, 'jfif_density': (1, 1), 'keywords': ['ok']}
Tags ajoutés à l'image : {'jfif': 257, 'jfif_version': (1, 1), 'jfif_unit': 0, 'jfif_density': (1, 1), 'keywords': ['ok']}
Tags ajoutés à l'image : {'keywords': ['ok']}
Tags ajoutés à l'image : {'jfif': 257, 'jfif_version': (1, 1), 'jfif_unit': 0, 'jfif_density': (1, 1), 'keywords': ['ok']}
Tags ajoutés à l'image : {'jfif': 257, 'jfif_version': (1, 1), 'jfif_unit': 0, 'jfif_density': (1, 1), 'keywords': ['ok']}
Tags ajoutés à l'image : {'jfif': 257, 'jfif_version': (1, 1), 'jfif_unit': 0, 'jfif_density': (1, 1), 'keywords': ['ok']}
Tags ajoutés à l'image : {'jfif': 257, 'jfif_version': (1, 1), 'jfif_unit': 0, 'jfif_density': (1, 1), 'keywords': ['ok']}
Tags ajoutés à l'image : {'keywords': ['ok']}
Error reading tags'keywords'
Error reading tags

In [64]:
from PIL import Image
import numpy
import math
import matplotlib.pyplot as plot
from sklearn.cluster import KMeans
import os
import json
import sys
import random
import requests
import shutil
import string
from pandas import json_normalize
import pandas as pd
import webcolors
import multiprocessing


# Chemin du dossier à parcourir
folder_path = "./images"

# Nombre de processus en parallèle
num_processes = 4

# Liste des propriétés de chaque image
images = []
numberOfCluster = 20


def addTagsImage(imgfile, file_name, tags):
    # Ajouter des tags
    imgfile.info["keywords"] = tags
        
    # Enregistrer l'image avec les tags
    imgfile.save(os.path.join(folder_path,file_name))
                
def clearTagsImage(imgfile, file_name):
    if "keywords" in imgfile.info:
        print(imgfile.info["keywords"])
        del imgfile.info["keywords"]

    # Enregistrer l'image sans tags
    imgfile.save(os.path.join(folder_path,file_name))

# Calcul de distance des couleurs
def color_distance(color1, color2):
    r1, g1, b1 = color1
    r2, g2, b2 = color2
    return math.sqrt((r1 - r2) ** 2 + (g1 - g2) ** 2 + (b1 - b2) ** 2)

def process(files):
    for filepath in files:
        if os.path.isfile(filepath):
            try :
                # Vérifie si le fichier est une image
                if filepath.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".gif")):
                    # Ouverture de l'image
                    with Image.open(filepath) as imgfile:
                        if "keywords" in imgfile.info:
                            print("KEYWORDS" + imgfile.info["keywords"])

                        file_name = os.path.basename(filepath)

                        tags = []
                        """
                        numarray = numpy.array(imgfile.getdata(), numpy.uint8)
                        # Effectuer le clustering avec numberOfCluster clusters
                        clusters = KMeans(n_clusters=numberOfCluster, n_init=2)
                        clusters.fit(numarray)

                        # Définir les limites des bins
                        npbins = numpy.arange(0, numberOfCluster+1)
                        histogram = numpy.histogram(clusters.labels_, bins=numpy.arange(0, numberOfCluster+1))

                        dominant_colors = clusters.cluster_centers_
                        frequency = histogram[0]
                        # trier les couleurs dominantes en fonction de leur fréquence décroissante
                        sorted_idx = numpy.argsort(frequency)[::-1]
                        sorted_colors = dominant_colors[sorted_idx]
                        hex_colors = []
                        # Parmi les 3 couleurs dominantes
                        for color in sorted_colors[:3]:
                            hex_color = '#%02x%02x%02x' % (
                                math.ceil(color[0]),
                                math.ceil(color[1]),
                                math.ceil(color[2])
                            )

                            # Récupérer une liste de couleurs prédéfinies
                            colors = webcolors.CSS3_HEX_TO_NAMES

                            # Initialiser les variables de distance minimale et de couleur la plus proche
                            min_distance = float('inf')
                            closest_color = None

                            # Parcourir chaque couleur prédéfinie et calculer sa distance avec la couleur de référence
                            for hex_code, name in colors.items():
                                rgb_code = webcolors.hex_to_rgb(hex_code)
                                distance = color_distance(color, rgb_code)

                                # Si la distance est plus petite que la distance minimale actuelle, mettre à jour la couleur la plus proche
                                if distance < min_distance:
                                    min_distance = distance
                                    closest_color = name

                            # Afficher la couleur la plus proche
                            tags.append(closest_color)
                        """
                        # Effacer les tags existants
                        clearTagsImage(imgfile, file_name)

                        # Enregistrement de l'image avec les tags
                        addTagsImage(imgfile, file_name, tags)
                        
            except Exception as e:
                print("Erreur sur l'image " + filepath)

processes = []
file_array_process = [[] for _ in range(num_processes)]
file_number = 0 

# Parcours des fichiers du dossier
for filename in os.listdir(folder_path):
    filepath = os.path.join(folder_path, filename)
    
    if os.path.isfile(filepath):
        try :
            # Vérifie si le fichier est une image
            if filepath.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".gif")):
                # Ouverture de l'image
                with Image.open(filepath) as imgfile:
                    file_name = os.path.basename(filepath)

                    tags = ["ok"]
                    """
                    numarray = numpy.array(imgfile.getdata(), numpy.uint8)
                    # Effectuer le clustering avec numberOfCluster clusters
                    clusters = KMeans(n_clusters=numberOfCluster, n_init=2)
                    clusters.fit(numarray)

                    # Définir les limites des bins
                    npbins = numpy.arange(0, numberOfCluster+1)
                    histogram = numpy.histogram(clusters.labels_, bins=numpy.arange(0, numberOfCluster+1))

                    dominant_colors = clusters.cluster_centers_
                    frequency = histogram[0]
                    # trier les couleurs dominantes en fonction de leur fréquence décroissante
                    sorted_idx = numpy.argsort(frequency)[::-1]
                    sorted_colors = dominant_colors[sorted_idx]
                    hex_colors = []
                    # Parmi les 3 couleurs dominantes
                    for color in sorted_colors[:3]:
                        hex_color = '#%02x%02x%02x' % (
                            math.ceil(color[0]),
                            math.ceil(color[1]),
                            math.ceil(color[2])
                        )

                        # Récupérer une liste de couleurs prédéfinies
                        colors = webcolors.CSS3_HEX_TO_NAMES

                        # Initialiser les variables de distance minimale et de couleur la plus proche
                        min_distance = float('inf')
                        closest_color = None

                        # Parcourir chaque couleur prédéfinie et calculer sa distance avec la couleur de référence
                        for hex_code, name in colors.items():
                            rgb_code = webcolors.hex_to_rgb(hex_code)
                            distance = color_distance(color, rgb_code)

                            # Si la distance est plus petite que la distance minimale actuelle, mettre à jour la couleur la plus proche
                            if distance < min_distance:
                                min_distance = distance
                                closest_color = name

                        # Afficher la couleur la plus proche
                        tags.append(closest_color)
                    """
                    # Effacer les tags existants
                    #clearTagsImage(imgfile, file_name)

                    # Enregistrement de l'image avec les tags
                    addTagsImage(imgfile, file_name, ["ok"])
                    
                    # Vérifier si les tags ont été ajoutés avec succès
                    print("Tags ajoutés à l'image :", imgfile.info.get("keywords"))

        except Exception as e:
            print("Erreur sur l'image " + filepath)
    """
    file_array_process[file_number % num_processes].append(filepath)
    file_number += 1

for i in range(len(file_array_process)):
    p = multiprocessing.Process(target=process, args=(file_array_process[i],))
    processes.append(p)
    p.start()

# Attendre que tous les processus soient terminés
for p in processes:
    p.join()
    """
    
    
def print_tags(folder_path):
    index = 0
    # Parcours des fichiers du dossier
    for filename in os.listdir(folder_path):
        filepath = os.path.join(folder_path, filename)

        # Vérifie si le fichier est une image
        if filepath.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".gif")):
            # Ouverture de l'image
            with Image.open(filepath) as imgfile:
                if "keywords" in imgfile.info:
                    # Vérifier si les tags ont été ajoutés avec succès
                    print(str(index) + "_Tags de l'image" + filename + " : " + imgfile.info.get("keywords"))
                else:
                    print(str(index) + "_Aucun tag trouvé pour l'image " + filename)
        index += 1
        
print_tags(folder_path)

0_Aucun tag trouvé pour l'image img_Kairo BW 1.jpg
1_Aucun tag trouvé pour l'image img_Amdavad Aerial.jpg
2_Aucun tag trouvé pour l'image img_Trieste (28766391880).jpg
3_Aucun tag trouvé pour l'image img_San Francisco from the Marin Headlands in March 2019.jpg
4_Aucun tag trouvé pour l'image img_Katowice Rynek.jpg
5_Aucun tag trouvé pour l'image img_Stuttgart Downtown Sights Collage.png
6_Aucun tag trouvé pour l'image img_SalernoCanalone.jpg
7_Aucun tag trouvé pour l'image img_00 sea towers (April 2018).jpg
8_Aucun tag trouvé pour l'image img_Stadtbild Kln (50MP).jpg
9_Aucun tag trouvé pour l'image img_-2015-(v2).jpg
10_Aucun tag trouvé pour l'image img_Omsk Collage 2016.png
11_Aucun tag trouvé pour l'image img_Tomsk0293.JPG
12_Aucun tag trouvé pour l'image img_NYC Downtown Manhattan Skyline seen from Paulus Hook 2019-12-20 IMG 7347 FRD (cropped).jpg
13_Aucun tag trouvé pour l'image img_Szczecin aerial 3a.jpg
14_Aucun tag trouvé pour l'image img_Russian church (37591925970).jpg
15_Aucu

In [94]:
from PIL import Image

# Ouverture de l'image
img = Image.open("./images/img_Turin Montage.png")

# Ajout de tags
img.info["auteur"] = "John Doe"
img.info["date"] = "02/04/2023"
# Enregistrement de l'image avec les tags en format PNG
img.save("./images/image_tags.png")

print(str(img.info))
# Ouverture de l'image
img2 = Image.open("./images/image_tags.png")
print(str(img2.info))

{'auteur': 'John Doe', 'date': '02/04/2023'}
{}
